# Imports

In [1]:
import pandas as pd
import numpy as np
 
# TensorFlow
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Concatenate
from tensorflow.keras.layers import Dropout, Dense, Lambda, Multiply, Subtract, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Activation, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam


# Scikit-learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# Text preprocessing
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Misc.
import os
import joblib
import random
import time
from tqdm import tqdm_notebook as tqdm
import pretty_midi

SEED = 42
%matplotlib inline

[nltk_data] Downloading package punkt to /home/naorko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/naorko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Load Data

### Load raw data

In [2]:
cols = ['Singer', 'Song Name', 'Lyrics']

df = pd.read_csv('datasets/lyrics_train_set.csv', names=cols)
df_test = pd.read_csv('datasets/lyrics_test_set.csv', names=cols)

#### Remove non rational lyrics

In [3]:
bad_song = df.query("Singer == 'darude'")
bad_song

,Singer,Song Name,Lyrics
315,darude,sandstorm,[instrumental] & du du dudududududuud & dududu...


In [4]:
df.drop(axis=0, index=bad_song.index, inplace=True)

### Add midi files' names

In [5]:
def add_midi_files(df):
    midis = list(os.listdir(r'./datasets/midi_files'))
    midis = {midi.lower()[:-4]: midi for midi in midis}    
    
    def combine_singer_song(singer, song):
        key = f'{singer} - {song}'.replace(' ', '_').lower()
        return midis[key] if key in midis else None
    
    df['Midi File'] = df.apply(lambda r: combine_singer_song(r['Singer'], r['Song Name']) ,axis=1)
    return df

In [6]:
df = add_midi_files(df)
df_test = add_midi_files(df_test)

#### check midi files

In [7]:
midis = np.concatenate([df['Midi File'].values, df_test['Midi File'].values])
midis.shape

(619,)

In [8]:
corrupted = []
for i, midi in enumerate(midis):
    try:
        midi = pretty_midi.PrettyMIDI(fr'./datasets/midi_files/{midi}')
        midi.remove_invalid_notes()
        del midi
    except Exception as e:
        print("%s\nerror readying midi file %s" % (e, midi))
        corrupted.append(i)

/home/naorko/.conda/envs/tf-env/lib/python3.8/site-packages/pretty_midi/pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Could not decode key with 16 sharps and mode 1
error readying midi file David_Bowie_-_Lazarus.mid
Could not decode key with 1 flats and mode 255
error readying midi file Beastie_Boys_-_Girls.mid
data byte must be in range 0..127
error readying midi file Billy_Joel_-_Movin'_Out.mid
data byte must be in range 0..127
error readying midi file Billy_Joel_-_Pressure.mid
Could not decode key with 4 flats and mode 255
error readying midi file Dan_Fogelberg_-_Leader_of_the_Band.mid

error readying midi file Brian_McKnight_-_On_The_Down_Low.mid
data byte must be in range 0..127
error readying midi file Aaron_Neville_-_Tell_It_Like_It_Is.mid


In [9]:
bad_songs = df.iloc[corrupted]
bad_songs

,Singer,Song Name,Lyrics,Midi File
91,david bowie,lazarus,look up here i'm in heaven & i've got scars th...,David_Bowie_-_Lazarus.mid
115,beastie boys,girls,girls all i really want is girls & and in the ...,Beastie_Boys_-_Girls.mid
136,billy joel,movin' out,anthony works in the grocery store & savin h...,Billy_Joel_-_Movin'_Out.mid
143,billy joel,pressure,you have to learn to pace yourself & pressure ...,Billy_Joel_-_Pressure.mid
189,dan fogelberg,leader of the band,an only child alone and wild a cabinet maker's...,Dan_Fogelberg_-_Leader_of_the_Band.mid
513,brian mcknight,on the down low,maxine was 5'9'' & had a man and she didn't mi...,Brian_McKnight_-_On_The_Down_Low.mid
575,aaron neville,tell it like it is,if you want something to play with & go and fi...,Aaron_Neville_-_Tell_It_Like_It_Is.mid


In [10]:
df.drop(axis=0, index=bad_songs.index, inplace=True)

### Create Validation Dataset

In [11]:
msk = np.random.rand(len(df)) < 0.8

df_train = df[msk]
df_val = df[~msk]

# Preprocessing

## Lyrics preprocessing

In [12]:
from nltk.tokenize import word_tokenize

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"in\'", "ing", phrase)
    phrase = re.sub(r"y\'all", "you all", phrase)
    
    # punctions
    regex = re.compile('[^a-zA-Z& ]')
    phrase = regex.sub('', phrase)
    
    return phrase

def preprocess_lyrics(data):
    data = decontracted(data)
    tokens = word_tokenize(data)
    data_arr = []
    
    for t in tokens:
        # Use only words, character combinations and numbers 
#         if not t.isalpha(): 
#             continue
            
        # Lower case word
        t = t.lower()
        
#         # Remove stop words
#         if t in sw: 
#             continue
        
        data_arr.append(t)
    
    
    return data_arr

In [13]:
df_train.iloc[1,2]

"winding your way down on baker street & lite in your head and dead on your feet & well another crazy day you drink the night away & and forget about everything & this city desert makes you feel so cold & its got so many people but its got no soul & and it's taken you so long to find out you were wrong & when you thought it held everything. & you used to think that it was so easy & you used to say that it was so easy & but you're tryin you're tryin now & another year and then you'd be happy & just one more year and then you'd be happy & but you're cryin' you're cryin' now & way down the street there's a light in his place & you open the door he's got that look on his face & and he asks you where you've been you tell him who you've seen & and you talk about anything & he's got this dream about buyin' some land & he's gonna give up the booze and the one night stands & and then he'll settle down it's a quiet little town & and forget about everything & but you know he'll always keep moving

In [14]:
string = df_train.iloc[1,2]
tokenized_string = preprocess_lyrics(string)

def pretty_lyrics(tokenized_string):
    for token in tokenized_string:
        if token == '&':
            print('\n')
        else:
            print(token, end=' ')

pretty_lyrics(tokenized_string)

winding your way down on baker street 

lite in your head and dead on your feet 

well another crazy day you drink the night away 

and forget about everything 

this city desert makes you feel so cold 

its got so many people but its got no soul 

and it is taken you so long to find out you were wrong 

when you thought it held everything 

you used to think that it was so easy 

you used to say that it was so easy 

but you are tryin you are tryin now 

another year and then you would be happy 

just one more year and then you would be happy 

but you are crying you are crying now 

way down the street there is a light in his place 

you open the door he is got that look on his face 

and he asks you where you have been you tell him who you have seen 

and you talk about anything 

he is got this dream about buying some land 

he is gon na give up the booze and the one night stands 

and then he will settle down it is a quiet little town 

and forget about everything 

but you know h

In [15]:
stop_token = '$'
lyrics_train = df_train['Lyrics'].apply(lambda s: preprocess_lyrics(s)[:-1] + [stop_token])
lyrics_val = df_val['Lyrics'].apply(lambda s: preprocess_lyrics(s)[:-1] + [stop_token])

### Create embeddings

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics_train)

In [17]:
lyrics_train = tokenizer.texts_to_sequences(lyrics_train)
lyrics_val = tokenizer.texts_to_sequences(lyrics_val)

In [18]:
EMBEDDING_FILE = './GoogleNews-vectors-negative300.bin'

if not os.path.isfile(EMBEDDING_FILE):
    !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
    !gzip -f -d GoogleNews-vectors-negative300.bin.gz

In [19]:
from gensim import models

embeddings_index = models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
embed_size = 300
word_index = tokenizer.word_index
max_features = len(word_index) + 1

nb_words = len(word_index)
embedding_matrix = (np.random.rand(nb_words+1, embed_size) - 0.5) / 5.0

not_in_word2vec = 0
for word, i in word_index.items():
    if i >= max_features: continue
    if word in embeddings_index:
        embedding_vector = embeddings_index.get_vector(word)
        embedding_matrix[i] = embedding_vector
    else:
        not_in_word2vec += 1
        
print(f'{not_in_word2vec} out of {len(word_index)} has no embedings from word2vec')

505 out of 6366 has no embedings from word2vec


#### Trying one word to whole song but one word

In [20]:
# train_x, train_y = [], []

# for lyric in lyrics:
#     for i in range(1, len(lyric)):
#         train_x.append(lyric[:i])
#         train_y.append(*lyric[i:i+1])
        
# train_x = pad_sequences(train_x)
# train_y = to_categorical(train_y)
# train_x.shape, train_y.shape

#### Trying sliding window of words

In [21]:
ast=np.lib.index_tricks.as_strided
def generate_sliding_window(arr, window_size=5, window_stride=1, last_window=False):
    last_window = 1 if last_window else 0
    arr = np.ascontiguousarray(arr)
    arr_len = arr.shape[0]
    s, = arr.strides
    windows_num = ((arr_len-window_size)//window_stride) + last_window
    
    return ast(arr, (windows_num, window_size), (s*window_stride, s))

In [22]:
def split_lyrics(lyrics, window_size=10):
    X, y = [], []
    for lyric in lyrics:
        X.append(generate_sliding_window(lyric, window_size))
        y.append(to_categorical(lyric[window_size:], num_classes=max_features))
        
    return X, y


In [23]:
window_size = 10
lyrics_train = split_lyrics(lyrics_train, window_size)
lyrics_val = split_lyrics(lyrics_val, window_size)

## Melody preprocessing

In [24]:
ast=np.lib.index_tricks.as_strided
def generate_sliding_window_2d(arr, window_size=5, window_stride=1, last_window=False):
    last_window = 1 if last_window else 0
    arr = np.ascontiguousarray(arr)
    l0, l1 = arr.shape
    s0, s1 = arr.strides
    windows_num = ((l0-window_size)//window_stride) + last_window
    
    return ast(arr, (windows_num, window_size, l1), (s0*window_stride, s0, s1))

In [25]:
def preprocess_melody_file_by_time(midi_name, fs=5, max_piano_val=100, fpw=4, window_size=10):
    pm = pretty_midi.PrettyMIDI(fr'./datasets/midi_files/{midi_name}')
    pm.remove_invalid_notes()
    
    # Sum all notes from all instruments
    piano_all = pm.get_piano_roll(fs=fs)
    piano_shape = piano_all.shape

    # Normalize by the number of played instruments in the same time and note
    counter = np.zeros(piano_shape)
    for inst in pm.instruments:
        curr_piano = inst.get_piano_roll(fs=fs)

        counter[:, :curr_piano.shape[1]] += (curr_piano > 0).astype(int)

    counter[counter == 0] = 1
    piano_all /= counter
    
    # Normalize by the maximum value of a note
    piano_all = piano_all / max_piano_val
    
    # Normalize by the number of played notes in the same time
    count_notes = (piano_all > 0).sum(axis=0)
    count_notes[count_notes == 0] = 1
    melody = piano_all.sum(axis=0) / count_notes
    del piano_all
    
    # 3 frames of 5fps equal to 0.6s ~ about one word
    melody_per_word = generate_sliding_window(melody[(melody > 0).argmax():], window_size=fpw, window_stride=(fpw*2)//5, last_window=True)
    melody_windows = generate_sliding_window_2d(melody_per_word, window_size=window_size, last_window=True)
    
    return melody_windows

In [32]:
midis = np.concatenate([df['Midi File'].values, df_test['Midi File'].values])
fs = 5

max_note_val = -1
min_frames = 9999
max_frames = -1
for midi in tqdm(midis):
    pm = pretty_midi.PrettyMIDI(fr'./datasets/midi_files/{midi}')
    pm.remove_invalid_notes()
    
    piano_roll = pm.get_piano_roll(fs=fs)
    if piano_roll.shape[1]:
        curr_len = piano_roll.shape[1]
        min_frames = min(min_frames, curr_len)
        max_frames = max(max_frames, curr_len)
        
    for inst in pm.instruments:
        piano_roll = inst.get_piano_roll(fs=fs)
        if piano_roll.shape[1]:
            curr_max_note = piano_roll.max()
            max_note_val = max(max_note_val, curr_max_note)
        
    del pm

print(f'The maximum note value is {max_note_val}')
print(f'The minimum song length is {min_frames}')
print(f'The maximum song length is {max_frames}')


The maximum note value is 762.0
The minimum song length is 297
The maximum song length is 3006


In [26]:
melody_train = df_train['Midi File'].apply(preprocess_melody_file_by_time, fs=100, max_piano_val=762, fpw=100, window_size=10)

/home/naorko/.conda/envs/tf-env/lib/python3.8/site-packages/pretty_midi/pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [ ]:
lyrics_train[0]

In [32]:
for i in range(len(lyrics_train[0])):
    mel = melody_train.iloc[i]
    lyr = lyrics_train[0][i]
    if mel.shape[0] >= lyr.shape[0]:
        melody_train.iloc[i] = mel[:lyr.shape[0], :, :]
    else:
        lyrics_train[0][i] = lyrics_train[0][i][:mel.shape[0], :]
        lyrics_train[1][i] = lyrics_train[1][i][:mel.shape[0], :]

In [33]:
np.concatenate(melody_train.values).shape

(141426, 10, 100)

In [34]:
np.concatenate(lyrics_train[0]).shape

(141426, 10)

In [35]:
np.concatenate(lyrics_train[1]).shape

(141426, 6367)

# Building the Lyrics Generator model

In [200]:
def init_simple(seq_len):
    inp = Input(shape=(seq_len,))
    
    embd = Embedding(max_features, 
                      embed_size, 
                      weights=[embedding_matrix],
                      input_length=seq_len,
                      name='word_embd')(inp)
    
    lstm = LSTM(100, return_sequences=True)(embd)
    lstm = LSTM(100)(lstm)

    X = Dense(100, activation="relu")(lstm)
    X = Dropout(0.5)(X)
    out = Dense(max_features, activation="softmax", name = 'out')(X)

    model = Model(inp, out)
    
#     model.get_layer('embd').trainable = False

    model.compile(loss='categorical_crossentropy', optimizer=Adam())
    
    return model

In [201]:
def get_callbacks(model_name):
    acc = 'val_loss'
    acc_mode = 'min'
    
    checkpoint = ModelCheckpoint(
                              fr'./models/{model_name}.h5', 
                              monitor=acc, 
#                               verbose=1, 
                              save_best_only=True, 
                              mode=acc_mode)
    earlystop = EarlyStopping(monitor=acc, mode=acc_mode, verbose=1, patience=6)
    reduceLR = ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', patience = 5,
                            factor = 0.5, min_lr = 1e-6, verbose = 1)

    return [checkpoint, reduceLR] #earlystop

In [203]:
def train_model(model_gen, train_data, val_data, use_saved=False, params_dict=None):
    os.makedirs('./models', exist_ok=True)
    params = ''
    if params_dict is not None:
        params = '_'.join(f'{key}_{val}' for key,val in params_dict.items())
    model_name = model_gen.__name__[5:] + f'_{params}'
        
    if use_saved:
        history = joblib.load(fr'./models/{model_name}_history.sav')
    else:
        callbacks = get_callbacks(model_name)
        
        train_x, train_y = train_data
        
        model = model_gen(train_x.shape[1]) # window size
        history = model.fit(
                            x=train_x,
                            y=train_y,
                            batch_size=params_dict['batch_size'],
                            epochs=params_dict['epochs'],
                            validation_data=val_data,
                            callbacks=callbacks,
                            verbose=1
                            )
        
        history = history.history
        joblib.dump(history, fr'./models/{model_name}_history.sav')
    
    model = load_model(fr'./models/{model_name}.h5')
    
    return model, history

In [204]:
model = init_simple(train_x.shape[1])
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 1204)]            0         
_________________________________________________________________
word_embd (Embedding)        (None, 1204, 300)         1913400   
_________________________________________________________________
lstm_17 (LSTM)               (None, 1204, 100)         160400    
_________________________________________________________________
lstm_18 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_10 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
out (Dense)                  (None, 6378)              644

In [205]:
params_dict = {'batch_size': 32, 'epochs': 20}
model, history= train_model(init_simple, train_data, val_data, use_saved=F, params_dict=params_dict)

Epoch 1/20
4619/4619 [==============================] - 86s 17ms/step - loss: 5.7913 - val_loss: 5.0301
Epoch 2/20
4619/4619 [==============================] - 70s 15ms/step - loss: 4.9867 - val_loss: 4.8607
Epoch 3/20
4619/4619 [==============================] - 69s 15ms/step - loss: 4.6565 - val_loss: 4.8774
Epoch 4/20
4619/4619 [==============================] - 76s 16ms/step - loss: 4.4195 - val_loss: 4.9408
Epoch 5/20
4619/4619 [==============================] - 76s 16ms/step - loss: 4.2456 - val_loss: 5.0937
Epoch 6/20
4619/4619 [==============================] - 77s 17ms/step - loss: 4.0573 - val_loss: 5.2618
Epoch 7/20
4619/4619 [==============================] - 76s 17ms/step - loss: 3.9183 - val_loss: 5.4201
Epoch 8/20
4619/4619 [==============================] - 76s 16ms/step - loss: 3.7498 - val_loss: 5.7651
Epoch 9/20
4619/4619 [==============================] - 75s 16ms/step - loss: 3.6126 - val_loss: 5.9113
Epoch 10/20
4619/4619 [==============================] - 77s 17m

# Sample Lyrics by Seed

In [ ]:
def generate_song(model, seed, window_size, stop_token, tokenizer, max_len):
    stop_token = tokenizer.word_index[stop_token]
    
    def get_next_word(seed):
        probs = model.predict(seed)
        chosen_idx = np.random.choice(range(0, max_features), p=probs[0])
        chosen_word = tokenizer.sequences_to_texts([[chosen_idx]])[0]
        
        return chosen_idx, chosen_word
    
    
    seed = preprocess_lyrics(seed)
    song = seed.copy()
    seed = ' '.join(seed)
    seed = tokenizer.texts_to_sequences([seed])
    seed = pad_sequences(seed, maxlen=window_size)

    i = 0
    idx, word = get_next_word(seed)
    while word != stop_token and i < max_len:
        song.append(word)
        i+=1
        seed = np.concatenate([seed[:,1:], [[idx]]], axis=1)
        idx, word = get_next_word(seed)
    
    return song    

In [ ]:
song_max_length = 2000
seed = 'Smack that all on the floor'
song = generate_song(model, seed, window_size, stop_token, tokenizer, song_max_length)
pretty_lyrics(song)